In [3]:
import json
import os
import pandas as pd
import numpy as np
from pathlib import Path
import collections
from sklearn.model_selection import train_test_split
from sklearn import metrics

import sys
sys.path.append("../")
from datatools.analyzer import *
from utterance.error_tools import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules import loss
import torch.optim as optim
import torch.nn.utils.rnn as rnn

In [5]:
with open("../../corpus/NTT/persona.json", "r", encoding="utf-8") as f:
    convs = json.load(f)

In [42]:
import random
random.seed(0)

def make_Xy(convs, n=4):

    X_str = []
    y = []

    all_utt = []
    for did in tqdm( convs["convs"] ) :
        dids = list( did.keys() )[0]
        all_utt += did[dids]
    random.shuffle(all_utt)

    j = 0

    for did in tqdm( convs["convs"] ):
        dids = list( did.keys() )[0]
        conv = did[dids]
        # print(conv)
        for i in range(n-1, len(conv)):
            p = (i-n+1)
            # print(i, "[{0}:{1}]".format(p, p+n), conv[p:p+n-1])
            # 正例
            if i%3 != 0:
                X_str.append( conv[p:p+n] )
                y.append(0)
                # print(i, conv[p:p+n])
            # 負例
            else:
                X_str.append( conv[p:p+n-1]+[all_utt[j]] )
                j += 1
                y.append(1)
    
    return X_str, y


In [43]:
X_str, y = make_Xy(convs, n=4)

100%|██████████| 5016/5016 [00:00<00:00, 206558.75it/s]


In [51]:
X_str[550:565]

[['それはどうかな。髪の毛痛んでるの？',
  'うん。ストレートで、ずっと伸ばしてるんだけど、枝毛が気になっちゃって。',
  'わたしもパーマかけてるから枝毛気になるのすごくわかる！',
  'そうなんだ！じゃあさ、豚骨ラーメンを1日3食食べて、髪の毛に効果があるか試してみてよ！'],
 ['うん。ストレートで、ずっと伸ばしてるんだけど、枝毛が気になっちゃって。',
  'わたしもパーマかけてるから枝毛気になるのすごくわかる！',
  'そうなんだ！じゃあさ、豚骨ラーメンを1日3食食べて、髪の毛に効果があるか試してみてよ！',
  '多分ないと思うけど試してみる！'],
 ['わたしもパーマかけてるから枝毛気になるのすごくわかる！',
  'そうなんだ！じゃあさ、豚骨ラーメンを1日3食食べて、髪の毛に効果があるか試してみてよ！',
  '多分ないと思うけど試してみる！',
  'わたし、宮城で生まれて、海鮮物をよく食べていたんですが、中でもウニが大好きなんです。'],
 ['前から思っていたんですけど、高校中退してるわたしと話してて退屈しませんか？',
  '全然ですよ。わたしも大学を途中で辞めちゃってますからね。似たようなものです！',
  'よかったです。しかもわたし鈍感だからいつのまにか人を不快にさせてることもあるみたいで。',
  'ありがとうございます。好物の寿司を食べられないのが地味に辛いです。'],
 ['全然ですよ。わたしも大学を途中で辞めちゃってますからね。似たようなものです！',
  'よかったです。しかもわたし鈍感だからいつのまにか人を不快にさせてることもあるみたいで。',
  'えー、そんなことないと思うけどなあ。そういうのって、誰かに言われたりする？',
  '遠回しに言われたことあるのかもしれないけど鈍感だからわかんなくて。だからあなたも思ったことがあったら遠慮なく言ってくださいね。'],
 ['よかったです。しかもわたし鈍感だからいつのまにか人を不快にさせてることもあるみたいで。',
  'えー、そんなことないと思うけどなあ。そういうのって、誰かに言われたりする？',
  '遠回しに言われたことあるのかもしれないけど鈍感だからわかんなくて。だからあなたも思ったことがあったら遠慮なく言ってくださいね。',
  'いや、わたしからは言